In [1]:
!cp -r /kaggle/input/paddy-full/* .

In [2]:
# yaml config for Yolo dataset
text = """
path: /kaggle/working/op
train: /kaggle/working/op/train
val: /kaggle/working/op/validate

nc: 12
names: [
  "blast",
  "black_stem_borer",
  "bacterial_panicle_blight",
  "bacterial_leaf_streak",
  "bacterial_leaf_blight",
  "hispa",
  "tungro",
  "brown_spot",
  "leaf_roller",
  "downy_mildew",
  "yellow_stem_borer",
  "white_stem_borer"
]
"""
yaml_file = "/kaggle/working/op/data.yaml"

with open(yaml_file,"w") as f:
  # print(text)
  f.write(text)

In [3]:
#Install Yolo v8
!pip install -U ultralytics "ray[tune]" --q

In [4]:
# !wandb disabled

In [5]:
!pip install -U ipywidgets -q

In [6]:
!pip install -U pydantic grpcio -q

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
apache-beam 2.46.0 requires dill<0.3.2,>=0.3.1.1, but you have dill 0.3.7 which is incompatible.
apache-beam 2.46.0 requires pyarrow<10.0.0,>=3.0.0, but you have pyarrow 11.0.0 which is incompatible.
tensorflowjs 4.16.0 requires packaging~=23.1, but you have packaging 21.3 which is incompatible.


In [7]:
from ultralytics import YOLO
from ray import tune

In [8]:
# Define a YOLO model
model = YOLO("yolov8m.pt")

100%|██████████| 49.7M/49.7M [00:00<00:00, 238MB/s]


In [9]:
space = {
    "lr0":tune.uniform(1e-5, 1e-2),
    "hsv_h":tune.uniform(0.0, 0.1),
    "hsv_s":tune.uniform(0.0, 0.9),
    "hsv_v":tune.uniform(0.0, 0.9),
    "degrees":tune.uniform(0.0, 45.0),
    "translate":tune.uniform(0.0, 0.9),
    "scale":tune.uniform(0.0, 0.9),
    "shear":tune.uniform(0.0, 10.0),
    "perspective":tune.uniform(0.0, 0.001),
    "flipud":tune.uniform(0.0, 1.0), 
    "fliplr":tune.uniform(0.0, 1.0),
    "mosaic":tune.uniform(0.0, 1.0),
    "mixup":tune.uniform(0.0, 1.0),
}

In [10]:
!pip uninstall wandb -y

Found existing installation: wandb 0.16.2
Uninstalling wandb-0.16.2:
  Successfully uninstalled wandb-0.16.2


In [11]:
import os
os.environ['WANDB_DISABLED'] = 'true'

In [12]:
!wandb off
!wandb disabled

/bin/bash: wandb: command not found
/bin/bash: wandb: command not found


In [13]:
!yolo checks

Ultralytics YOLOv8.1.9 🚀 Python-3.10.13 torch-2.1.2 CUDA:0 (Tesla T4, 15102MiB)
Setup complete ✅ (4 CPUs, 31.4 GB RAM, 5432.3/8062.4 GB disk)

OS                  Linux-5.15.133+-x86_64-with-glibc2.31
Environment         Kaggle
Python              3.10.13
Install             pip
RAM                 31.36 GB
CPU                 Intel Xeon 2.00GHz
CUDA                12.1

WARNING ⚠️ invalid check_version(None, >=3.3.0) requested, please check values.
matplotlib          ✅ None>=3.3.0
numpy               ✅ 1.26.3>=1.22.2
opencv-python       ✅ 4.9.0.80>=4.6.0
pillow              ✅ 9.5.0>=7.1.2
pyyaml              ✅ 6.0.1>=5.3.1
requests            ✅ 2.31.0>=2.23.0
scipy               ✅ 1.12.0>=1.4.1
torch               ✅ 2.1.2>=1.8.0
torchvision         ✅ 0.16.2>=0.9.0
tqdm                ✅ 4.66.1>=4.64.0
psutil              ✅ 5.9.7
py-cpuinfo          ✅ 9.0.0
thop                ✅ 0.1.1-2209072238>=0.1.1
pandas              ✅ 2.2.0>=1.1.4
seaborn             ✅ 0.12.2>=0.11.0


In [14]:
# Run Ray Tune on the model
result_grid = model.tune(data="/kaggle/working/op/data.yaml",
                         space=space,
                         epochs=10,
                         gpu_per_trial=1,
                         use_ray=True)

(_tune pid=405) Ultralytics YOLOv8.1.9 🚀 Python-3.10.13 torch-2.1.2 CUDA:0 (Tesla T4, 15102MiB)
(_tune pid=405) engine/trainer: task=detect, mode=train, model=yolov8m.pt, data=/kaggle/working/op/data.yaml, epochs=10, time=None, patience=50, batch=16, imgsz=640, save=True, save_period=-1, cache=False, device=None, workers=8, project=None, name=train, exist_ok=False, pretrained=True, optimizer=auto, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, vid_stride=1, stream_buffer=False, visualize=False, augment=False, agnostic_nms=False, classes=None, retina_masks=False, embed=None, show=False, save_frames=False, save_txt=False, save_conf=False, save_crop=False, show_labels=T

100%|██████████| 755k/755k [00:00<00:00, 26.0MB/s]
(_tune pid=405) 2024-02-03 20:27:16.490449: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
(_tune pid=405) 2024-02-03 20:27:16.490550: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
(_tune pid=405) 2024-02-03 20:27:16.648799: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


(_tune pid=405) Overriding model.yaml nc=80 with nc=12
(_tune pid=405) 
(_tune pid=405)                    from  n    params  module                                       arguments                     
(_tune pid=405)   0                  -1  1      1392  ultralytics.nn.modules.conv.Conv             [3, 48, 3, 2]                 
(_tune pid=405)   1                  -1  1     41664  ultralytics.nn.modules.conv.Conv             [48, 96, 3, 2]                
(_tune pid=405)   2                  -1  2    111360  ultralytics.nn.modules.block.C2f             [96, 96, 2, True]             
(_tune pid=405)   3                  -1  1    166272  ultralytics.nn.modules.conv.Conv             [96, 192, 3, 2]               
(_tune pid=405)   4                  -1  4    813312  ultralytics.nn.modules.block.C2f             [192, 192, 4, True]           
(_tune pid=405)   5                  -1  1    664320  ultralytics.nn.modules.conv.Conv             [192, 384, 3, 2]              
(_tune pid=405)   

  0%|          | 0.00/6.23M [00:00<?, ?B/s]
100%|██████████| 6.23M/6.23M [00:00<00:00, 116MB/s]


(_tune pid=405) AMP: checks passed ✅


...

                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 8/8 [00:03<00:00,  2.64it/s]


(_tune pid=405) 
(_tune pid=405) 10 epochs completed in 0.122 hours.
(_tune pid=405) Optimizer stripped from runs/detect/train/weights/last.pt, 52.0MB
(_tune pid=405) Optimizer stripped from runs/detect/train/weights/best.pt, 52.0MB
(_tune pid=405) 
(_tune pid=405) Validating runs/detect/train/weights/best.pt...
(_tune pid=405) Ultralytics YOLOv8.1.9 🚀 Python-3.10.13 torch-2.1.2 CUDA:0 (Tesla T4, 15102MiB)
(_tune pid=405) Model summary (fused): 218 layers, 25846708 parameters, 0 gradients, 78.7 GFLOPs


                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95):   0%|          | 0/8 [00:00<?, ?it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95):  12%|█▎        | 1/8 [00:00<00:02,  2.44it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95):  25%|██▌       | 2/8 [00:01<00:03,  1.92it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95):  38%|███▊      | 3/8 [00:02<00:04,  1.07it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95):  50%|█████     | 4/8 [00:02<00:03,  1.33it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95):  62%|██████▎   | 5/8 [00:03<00:02,  1.33it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95):  75%|███████▌  | 6/8 [00:04<00:01,  1.63it/s]
                 Class     Images  Instances      Box(P 

(_tune pid=405)                    all        250        345      0.604      0.116      0.114     0.0449
(_tune pid=405)                  blast        250          7          1          0     0.0118    0.00271
(_tune pid=405)       black_stem_borer        250         17      0.212      0.235      0.157     0.0447
(_tune pid=405) bacterial_panicle_blight        250         28          1          0     0.0265    0.00672
(_tune pid=405)  bacterial_leaf_streak        250         44     0.0798     0.0682     0.0633     0.0234
(_tune pid=405)  bacterial_leaf_blight        250         24          0          0     0.0327    0.00888
(_tune pid=405)                  hispa        250         74      0.264     0.0676     0.0855      0.027
(_tune pid=405)                 tungro        250         21          1          0     0.0219    0.00652
(_tune pid=405)             brown_spot        250         30          1          0     0.0282     0.0099
(_tune pid=405)            leaf_roller        250    

2024-02-03 21:44:11,621	INFO tune.py:1042 -- Total run time: 4634.40 seconds (4625.45 seconds for the tuning loop).


In [15]:
!ls /tmp/ray/session_latest/logs/

agent-424238335.err
agent-424238335.out
dashboard.err
dashboard.log
dashboard_agent.log
debug_state.txt
debug_state_gcs.txt
events
gcs_server.err
gcs_server.out
log_monitor.err
log_monitor.log
monitor.err
monitor.log
monitor.out
nsight
old
python-core-driver-01000000ffffffffffffffffffffffffffffffffffffffffffffffff_26.log
python-core-worker-075ef8d3b47e9e33277284e2cf8e9d6b8bb1242dc021c18aca5008dc_222.log
python-core-worker-256cf07955381a121d1811db65f9c63293c3b49d48d0b57a1db9dbc5_223.log
python-core-worker-4ccda1e6bfb95c058d4d26a34dc6468694a544bd1e67262d88b1b0f2_405.log
python-core-worker-9b8da89ff2d2f4995cdbee61af073295bf9cf47e0eb7057aecac7b79_224.log
python-core-worker-d732f38086bfd41c5813a324db02c09cf52ceca8e61a45c0f4633f60_221.log
raylet.err
raylet.out
runtime_env_agent.err
runtime_env_agent.log
runtime_env_agent.out
runtime_env_setup-01000000.log
worker-075ef8d3b47e9e33277284e2cf8e9d6b8bb1242dc021c18aca5008dc-ffffffff-222.err
worker-075ef8d3b47e9e33277284e2cf8e9d6b8bb1242dc021c18aca